In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [2]:
data = pd.read_csv('/data/data/merged_transr_final_data1.csv')

In [4]:
data.shape

(33737508, 54)

In [5]:
movie_sth = pd.read_csv('/data/data/Movie.txt',sep='\t',encoding='utf-8')

In [6]:
all_movie_info1 = pd.merge(data,movie_sth,on=['mid'],how='left')

In [7]:
all_movie_info1.head()

,uid,mid,star,timeStamp,r_1,r_2,r_3,r_4,r_5,r_6,...,duration,releasedTime,tags,ratingCount,star5,star4,star3,star2,star1,language
0,1722994,1306505,1,2007.0,0.052603,-0.040752,-0.056807,0.020781,0.150330,0.207236,...,91.0,1988-02(柏林电影节);,张艺谋(11433);姜文(6883);巩俐(5296);中国电影(4393);大陆(273...,40137.0,25.4,53.9,19.3,1.2,0.3,汉语普通话
1,1405477,10574468,1,2013.0,-0.002572,0.004976,0.003739,0.005666,-0.012067,-0.021188,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15849871,4910186,1,2011.0,-0.065039,0.064973,0.177405,0.169921,0.090022,0.205181,...,0.0,NaN,BONES(404);动画(273);宮野真守(249);日本动画(197);日本(174)...,1547.0,31.1,36.2,24.7,4.7,3.4,日语
3,1068524,1304643,1,2006.0,-0.052975,-0.076582,-0.099915,0.147024,0.040548,0.191201,...,92.0,2001-07-18(美国);,科幻(4907);美国(3174);侏罗纪公园(2367);斯皮尔伯格(2136);美国电影...,24127.0,8.8,32.4,50.8,6.9,1.1,英语
4,1307041,1851857,1,2008.0,-0.126394,0.047633,-0.152797,-0.107490,0.240108,0.284580,...,152.0,2008-07-18(美国);,美国(17483);科幻(16210);动作(12058);犯罪(6699);诺兰(6670...,123135.0,53.4,33.4,11.7,1.2,0.3,英语


In [11]:
all_movie_info1.to_csv('/data/data/all_movie_info_1.csv')

In [8]:
for i in all_movie_info1.select_dtypes('object').columns:
    lbl = LabelEncoder()
    all_movie_info1[i] = lbl.fit_transform(all_movie_info1[i].fillna(-1).astype(str))

In [9]:
from sklearn.cross_validation import train_test_split
feature_name = [i for i in all_movie_info1 if i not in ['star']]
train_data,valid_data,train_label,valid_label = train_test_split(all_movie_info1[feature_name],all_movie_info1['star'],random_state=1998,test_size=0.2)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [10]:
import lightgbm as lgb
dtrain = lgb.Dataset(train_data[feature_name], label=train_label.values)
dval = lgb.Dataset(valid_data[feature_name], label=valid_label.values)

params = {'learning_rate': 0.1,
          'metric': ['auc','binary_logloss'],
          'objective': 'binary',
          'nthread': 32,
          'num_leaves': 16,
          'colsample_bytree': 0.9,
          'bagging_fraction' : 0.9,
          'bagging_freq' : 10,
          'seed' : 2018,
        }
         
lgb_model = lgb.train(params, dtrain, 1000, dval, verbose_eval=10,early_stopping_rounds=100,)

MemoryError: 

In [11]:
lgb_model.save_model('model-transR.txt')

### test